## Вопросно-ответные текста и понимание текста

QA - одна из самых модных тем в Nlp. Часто ещё используют термин reading comprehension (понимание текста), но разницу понять очень сложно. За последнее время вышло много вопросно-ответных датасетов: 

Другие задачи тоже уже пробуют переделывать под формат вопросно-ответных систем.

Самый популярный датасет - SQUAD от Стэнфорда. На нем тестируют все самые новые нейронки (BERT например). 

Давайте попробуем обучить какую-нибудь нейронку на этих данных.

In [1]:
import json
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Dropout, Dense, Activation, CuDNNLSTM
from keras.layers import LSTM, Bidirectional,Input
from keras.layers import concatenate
from keras.models import Sequential
from keras import optimizers
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Conv1D, MaxPooling1D, Dropout, GlobalAveragePooling1D, LSTM
from keras import backend as K
from keras.layers.core import Layer  
from keras import initializers, regularizers, constraints  
from keras import backend as K

from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import keras
import gensim

Using TensorFlow backend.


In [2]:
train = json.load(open('train-v2.0.json'))

In [3]:
dev = json.load(open('dev-v2.0.json'))

Не будем заморачиваться с нормализацией.

In [4]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word and word]

    return words

Датасет устроен так - есть тексты из википедии, к какому-то параграфу этого текста задан вопрос и из этого же параграфа извлечен ответ. В версии 2.0 добавились также вопросы, на которые нет ответа, что усложняет задачу для модели, но мы пока будет игнорировать такие вопросы.

Для каждого ответа даны начало и конец (индексы) в соответствующем параграфе. На этих индексах мы и будем обучаться.

Но для начала нужно все предобработать. Чтобы скоратить время обучения предобучим fastext на всех параграфах и вопросах (можно взять готовую модель получше).

In [5]:
corpus = []

In [6]:
contexts = []
questions = []

starts = []
ends = []

for instance in train['data']:
    for paragraph in instance['paragraphs']:
        context = paragraph['context']
        corpus.append(normalize(context))
        
        for qas in paragraph['qas']:
            question = qas['question']
            
            if qas['is_impossible']:
                continue
            
            for answer in qas['answers']:
                start = answer['answer_start']
                end = len(answer['text'])
                contexts.append(normalize(context))
                questions.append(normalize(question))
                starts.append(start)
                ends.append(end)

                corpus.append(normalize(question))

Для отложенной выборки сохраним исходные параграфы и индексы вопросов. Они пригодятся для тестирования.

In [7]:
contexts_dev = []
questions_dev = []

starts_dev = []
ends_dev = []
ids = []
imporssible_ids = []
raw_context = []
for instance in dev['data']:
    for paragraph in instance['paragraphs']:
        context = paragraph['context']
        corpus.append(normalize(context))
        for qas in paragraph['qas']:
            if qas['is_impossible']:
                imporssible_ids.append(qas['id'])
                continue
            
            question = qas['question']
            
            for answer in qas['answers']:
                
                start = answer['answer_start']
                end = len(answer['text'])
                contexts_dev.append(normalize(context))
                questions_dev.append(normalize(question))
                starts_dev.append(start)
                ends_dev.append(end)
                corpus.append(normalize(question))
                ids.append(qas['id'])
                raw_context.append(context)

Обучаем фастекст.

In [8]:
ft = gensim.models.FastText(corpus, size=200, sg=1)

Теперь построим словарь.

In [9]:
vocab = set()

for context in contexts:
    vocab.update(context)

for question in questions:
    vocab.update(question)
    
for context in contexts_dev:
    vocab.update(context)

for question in questions_dev:
    vocab.update(question)

id2word = {i+1:word for i, word in enumerate(vocab)}
word2id = {word:i for i, word in id2word.items()}

Создадим матрицу со эмбеддингами всех слова. Потом подадим её к Embedding слой нейронки.

In [10]:
embeddings = np.zeros((len(vocab) + 1, 200))

for i in range(1, len(vocab)+1):
    try:
        embeddings[i] = ft[id2word[i]]
    except KeyError:
        embeddings[i] = np.random.rand((200))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


Теперь преодразуем все слова индексы и привидем все к одной длине (максимальной).

In [11]:
## КОНТЕКСТ

In [12]:
contexts_le = [[word2id[word] for word in context] for context in contexts]
max_len = max([len(c) for c in contexts])

X_train_context = pad_sequences(contexts_le, max_len, padding='post')

In [13]:
contexts_le_dev = [[word2id.get(word, 0) for word in context] for context in contexts_dev]

X_dev_context = pad_sequences(contexts_le_dev, max_len, padding='post')

In [14]:
## ВОПРОС

In [15]:
questions_le = [[word2id[word] for word in question] for question in questions]
max_len_q = max([len(c) for c in questions])

X_train_question = pad_sequences(questions_le, max_len_q, padding='post')

In [16]:
questions_le_dev = [[word2id.get(word, 0) for word in question] for question in questions_dev]
X_dev_question = pad_sequences(questions_le_dev, max_len_q, padding='post')

Зададим параметры для нейронки.

In [17]:
vocab_size = len(vocab)+1
embedding_vector_length = 200

max_span_begin = np.max(starts)
max_span_end = np.max(ends)


In [18]:
starts = np.array(starts)  
ends = np.array(ends) 

starts_dev = np.array(starts_dev) 
ends_dev = np.array(ends_dev)

Теперь самая сложная часть. У нашей нейронки будет два входа и два выхода. 

**Первый вход** - для параграфа. Он будет эбмедиться и прогоняться через LSTM.

**Второй вход** - для вопроса. Он будет эбмедиться и прогоняться через LSTM.

**Конкатенация** - выходы с двух входов скливаются в один. 

**Первый выход** - для начального индекса. Тут будет классификация из N классов, где N - это максимальный индекс в ответах (на самом деле честнее делать просто длину параграфа)

**Второй выход** - для длинны ответа. Тут будет классификация из N классов, где N - это максимальная длина ответа (на самом деле честнее делать просто длину параграфа)

Так как мы решаем задачу классификации - лосс **categorical_crossentropy** (sparse потому что мы не энкодили в ohe вектора)

In [23]:


# Первый вход
context_input = Input(shape=(max_len, ), name='context_input')
emb_c = Embedding(input_dim=vocab_size, output_dim=200, weights=[embeddings], 
              input_length=max_len, trainable=False)(context_input)

lstm_out_c = Bidirectional(CuDNNLSTM(50,  return_sequences=True,))(emb_c)
drop_1 = Dropout(0.1)(lstm_out_c)

# Второй вход
ques_input = Input(shape=(max_len_q, ), name='ques_input')
emb_q = Embedding(input_dim=vocab_size, output_dim=200, weights=[embeddings], 
              input_length=max_len_q, trainable=False)(ques_input)
lstm_out_q = Bidirectional(CuDNNLSTM(50,return_sequences=True,) )(emb_q)
drop_2 = Dropout(0.1)(lstm_out_q)

# merger model
merge_layer = concatenate([drop_1, drop_2], axis=1)
biLSTM_s = Bidirectional(CuDNNLSTM(10,))(merge_layer)
biLSTM_e = Bidirectional(CuDNNLSTM(10,))(merge_layer)

# Выход 1
softmax_1 = Dense(max_span_begin+1, activation='softmax', name='start')(biLSTM_s)

# Выход 2
softmax_2 = Dense(max_span_end+1, activation='softmax', name='end')(biLSTM_e)

model = Model(inputs=[context_input, ques_input], outputs=[softmax_1, softmax_2])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_input (InputLayer)      (None, 650)          0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 650, 200)     22157600    context_input[0][0]              
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 40, 200)      22157600    ques_input[0][0]                 
__________________________________________________________________________________________________
bidirectio

Попробуем обучаться.

In [24]:
validation_data=({'context_input': X_dev_context,
                  'ques_input':X_dev_question}, 
                 {'start': starts_dev,
                  'end': ends_dev})


training_data=({'context_input': X_train_context,
                'ques_input':X_train_question}, 
                 {'start': starts,
                  'end': ends})

model.fit(training_data[0], training_data[1], batch_size=1024,  epochs=100, shuffle=True,
          validation_data=(validation_data[0], validation_data[1]))

Train on 86821 samples, validate on 20302 samples
Epoch 1/100
86821/86821 [==============================] - 51s 591us/step - loss: 12.0245 - start_loss: 7.4201 - end_loss: 4.6044 - start_acc: 0.0118 - end_acc: 0.0347 - val_loss: 10.9409 - val_start_loss: 6.9387 - val_end_loss: 4.0022 - val_start_acc: 0.0230 - val_end_acc: 0.0428
Epoch 2/100
86821/86821 [==============================] - 50s 577us/step - loss: 10.7175 - start_loss: 6.7949 - end_loss: 3.9226 - start_acc: 0.0256 - end_acc: 0.0769 - val_loss: 10.7246 - val_start_loss: 6.8200 - val_end_loss: 3.9046 - val_start_acc: 0.0230 - val_end_acc: 0.0735
Epoch 3/100
86821/86821 [==============================] - 50s 578us/step - loss: 10.6130 - start_loss: 6.7294 - end_loss: 3.8836 - start_acc: 0.0256 - end_acc: 0.0817 - val_loss: 10.6893 - val_start_loss: 6.8017 - val_end_loss: 3.8876 - val_start_acc: 0.0230 - val_end_acc: 0.0735
Epoch 4/100
28672/86821 [========>.....................] - ETA: 30s - loss: 10.6012 - start_loss: 6.7158

KeyboardInterrupt: 

Так как метрики не очень подходящие можно оценить все с помощью evaluation скрипта от SQuAd.

In [ ]:
preds = model.predict(validation_data[0], verbose=1)

Предиктим распределения для каждого выхода. Делаем argmax() чтобы достать самый вероятный класс (индекс).

In [ ]:
starts_pred = preds[0].argmax(axis=1)
ends_pred = preds[1].argmax(axis=1)

По индексам достаем текст из нетронутых текстов из отложенной выборки. Для плохих вопросов, просто добавляем пустые строки (что на самом деле нечестно).

In [ ]:
pred_dict = {}
for i in range(len(starts_pred)):
    cont = raw_context[i]
    span = cont[starts_pred[i]:starts_pred[i]+ends_pred[i]]
    pred_dict[ids[i]] = span

for idx in imporssible_ids:
    pred_dict[idx] = ""

In [ ]:
pred_dict

In [ ]:
import json
json.dump(pred_dict, open('prediction.json', 'w'),)

Запускаем скрипт.

In [ ]:
!python3 evaluate-v2.0.py dev-v2.0.json prediction.json

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Conv1D, MaxPooling1D, Dropout, GlobalAveragePooling1D, LSTM
from keras import backend as K
from keras.layers.core import Layer  
from keras import initializers, regularizers, constraints  
from keras import backend as K

from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]